In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-sweep-new/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[1]

269 experiments


{'commit': '70e0736190637a9d66177214260574d22855f4b6',
 'id': '9a3cd6b0e65e36e00c44ff6721ee5a8884805f22',
 'name': 'owned-dole',
 'params': {'dataset': {'path': 'bdsaglam/musique-sweep',
   'name': 'answerable',
   'split': 'train'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.0,
   'system_prompt': 'cte/format-triples-ere-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'semi/cte-triples.json',
   'n_shot': 2,
   'n_sc': 1},
  'run': 1},
 'metrics': {}}

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-sweep")
    & (df["metrics.f1"] > 0.1)
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

136 experiments after preprocessing


In [4]:
def parse_qa_technique(row):
    base_technique = row["params.qa.system_prompt"].split("/")[0].upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [5]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.n_sc",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.n_sc,params.qa.temperature,params.run
0,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,1,0.0,"(1, 2)"
1,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,1,0.1,"(2,)"
2,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,1,1.0,"(2,)"
3,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,1,0.0,"(1,)"
4,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,1,0.5,"(1,)"
...,...,...,...,...,...,...,...
92,direct/format-few.txt,cq.txt,semi/direct.json,3,1,0.0,"(1, 2)"
93,direct/format-few.txt,cq.txt,semi/direct.json,3,1,0.1,"(1,)"
94,direct/format-few.txt,cq.txt,semi/direct.json,3,1,0.5,"(1,)"
95,direct/format-few.txt,cq.txt,semi/direct.json,3,1,0.7,"(1, 2)"


In [6]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	cok/adapted.txt
	cot/format-thought.txt
	cte/format-triples-ere-few.txt
	cte/format-triples-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	semi/ccot.json
	semi/cok.json
	semi/cot.json
	semi/cte-triples.json
	semi/direct.json
- params.qa.n_shot
	0
	1
	2
	3
- params.qa.n_sc
	1
- params.qa.temperature
	0.0
	0.1
	1.0
	0.5
	0.7
- params.run
	(1, 2)
	(2,)
	(1,)


In [7]:
df.to_json('exps.jsonl', orient='records', lines=True)